## Imports

In [2]:
from tqdm.notebook import tqdm  # Import tqdm for Jupyter Notebook

from src.optimizee import *
from src.optimizer import *
from src.initializer import *

from src.train_lstm import *
from src.test_optimizer import *

from src.interpretable_train_lstm import *
from src.preprocessed_train_lstm import *

import shutil
import time
import matplotlib.pyplot as plt

from torch.utils.tensorboard import SummaryWriter

## Sanity Runs

In [2]:
torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)
n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01, 0.02]}

initializer = Param_Initializer(QuadraticOptimizee, kwargs)
print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=10, min_horizon=50, max_horizon=50, discount=2)
params = test_LSTM(lstm_optimizer, initializer, time_horizon=50)

2


Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Epoch [1/10], Cumulative Loss: 334928559128707072.0000, LR: 1.000e-03
Epoch [2/10], Cumulative Loss: 223858282492592128.0000, LR: 1.000e-03
Epoch [3/10], Cumulative Loss: 124885124843044864.0000, LR: 1.000e-03
Epoch [4/10], Cumulative Loss: 97719551185125376.0000, LR: 1.000e-03
Epoch [5/10], Cumulative Loss: 97970171116781568.0000, LR: 1.000e-03
Epoch [6/10], Cumulative Loss: 115990866048319488.0000, LR: 1.000e-03
Epoch [7/10], Cumulative Loss: 63587291464990720.0000, LR: 1.000e-03
Epoch [8/10], Cumulative Loss: 176612559904833536.0000, LR: 1.000e-03
Epoch [9/10], Cumulative Loss: 138666910391730176.0000, LR: 1.000e-03
Epoch [10/10], Cumulative Loss: 97805201422942208.0000, LR: 1.000e-03

Training complete!
Final Loss: 848.5136108398438


In [3]:
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()
X, y = diabetes.data, diabetes.target
y = (y-np.mean(y))/np.std(y)
X.shape, y.shape

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss(), nn.L1Loss()]}

lstm_optimizer = LSTMConcurrent(num_optims=2, preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)
initializer = Param_Initializer(XYNNOptimizee, kwargs)

lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=10, min_horizon=50, max_horizon=50, discount=0)
params = test_LSTM(lstm_optimizer, initializer, time_horizon=50)

Training Progress:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Cumulative Loss: 3201211.7500, LR: 1.000e-03
Epoch [2/10], Cumulative Loss: 2120136.2500, LR: 1.000e-03
Epoch [3/10], Cumulative Loss: 1421280.3750, LR: 1.000e-03
Epoch [4/10], Cumulative Loss: 1140170.0000, LR: 1.000e-03
Epoch [5/10], Cumulative Loss: 871803.2500, LR: 1.000e-03
Epoch [6/10], Cumulative Loss: 617131.6250, LR: 1.000e-03
Epoch [7/10], Cumulative Loss: 469041.4688, LR: 1.000e-03
Epoch [8/10], Cumulative Loss: 395526.3125, LR: 1.000e-03
Epoch [9/10], Cumulative Loss: 310691.6875, LR: 1.000e-03
Epoch [10/10], Cumulative Loss: 223572.9375, LR: 1.000e-03

Training complete!
Final Loss: 136564.59375


## Quadratic Optimizee

In [4]:
shutil.rmtree("Quadratic/Optimizers/Adam", ignore_errors=True)

torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)

n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

optimizee = QuadraticOptimizee(W, theta0)
optimizee.set_params()
optimizer_cls = optim.Adam
optimizer_kwargs = {"lr":0.05}

writer = SummaryWriter(f"Quadratic/Optimizers/Adam")
test_optimizer(optimizer_cls, optimizee,optimizer_kwargs, time_horizon=500, writer=writer)

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Training Progress:   0%|          | 0/500 [00:00<?, ?time step/s]

tensor([[1.3852],
        [0.5164],
        [1.5571],
        [0.7063],
        [0.7442],
        [0.9559],
        [0.1205],
        [1.2055],
        [0.9906],
        [0.7883]], requires_grad=True)

In [5]:
shutil.rmtree("Quadratic/Optimizers/LSTM", ignore_errors=True)

torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)

n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}
initializer = Param_Initializer(QuadraticOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=200, max_horizon=500, discount=0)
writer = SummaryWriter(f"Quadratic/Optimizers/LSTM")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Epoch [10/100], Cumulative Loss: 18863.7344, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.0161, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 9.6801, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0483, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0059, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0231, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0157, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0169, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0197, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0495, LR: 1.000e-03

Training complete!
Final Loss: 0.025036029517650604


In [6]:
for i in range(4):
    th = i * 100
    shutil.rmtree(f"Quadratic/Horizon/{th}", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)
    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500-th, max_horizon=500+th, discount=0)
    writer = SummaryWriter(f"Quadratic/Horizon/{th}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=1000, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Epoch [10/100], Cumulative Loss: 87100.5781, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.3382, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.1890, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0665, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.3615, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.3239, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.1530, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0884, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.3373, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.2057, LR: 1.000e-03

Training complete!
Final Loss: 0.16304242610931396


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 82497.3750, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.2789, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.1875, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.2674, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.2013, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.3278, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.2189, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0411, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.1276, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.5433, LR: 1.000e-03

Training complete!
Final Loss: 0.49988266825675964


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 136763.3594, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.2124, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.1286, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.1214, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.1698, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.1458, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.3471, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.1434, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.1870, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.1961, LR: 1.000e-03

Training complete!
Final Loss: 0.2404858022928238


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 38981.0156, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 754.7361, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.1826, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.1219, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.1317, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0988, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.3531, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0957, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0760, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.1229, LR: 1.000e-03

Training complete!
Final Loss: 0.1588994711637497


In [23]:
for discount in [0, 1e-3, 0.1, 0.9, 1]:
    
    shutil.rmtree(f"Quadratic/Discount/Train_{discount}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Quadratic/Discount/Test_{discount}", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)
    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"Quadratic/Discount/Train_{discount}")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=discount, writer=writer)
    writer = SummaryWriter(f"Quadratic/Discount/Test_{discount}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Epoch [10/100], Cumulative Loss: 19161.7090, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1679.9042, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0424, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0149, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0937, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.1046, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.1442, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.3084, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.1321, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.1007, LR: 1.000e-03

Training complete!
Final Loss: 0.14113982021808624
Final parameters: [[ 1.8663387  -0.18337986  2.13553     0.29716218  0.44743246  0.79809546
  -0.9425076   1.343382    0.7459527   0.5683814 ]]...


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 19180.8203, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1743.2588, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0289, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0382, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0766, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.1223, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.1372, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.2658, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.1685, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0767, LR: 1.000e-03

Training complete!
Final Loss: 0.10835688561201096
Final parameters: [[ 1.9424584  -0.22430722  2.132454    0.29362375  0.4429594   0.79527706
  -0.936571    1.3651272   0.74557936  0.5520531 ]]...


...

Epoch [10/100], Cumulative Loss: 21282.2812, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1922.0547, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0245, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0248, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0975, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.1307, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.1743, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.2652, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.1349, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.1239, LR: 1.000e-03

Training complete!
Final Loss: 0.15897709131240845
Final parameters: [[ 2.0037286  -0.35737175  2.2502873   0.20876706  0.43233424  0.74677634
  -1.177658    1.3873047   0.8225865   0.50722057]]...


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 185512.4844, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 16820.3613, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.3394, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.2994, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.6529, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.1399, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 1.7025, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 2.2102, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.1801, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.6545, LR: 1.000e-03

Training complete!
Final Loss: 0.1259104311466217
Final parameters: [[ 2.0240297  -0.30898386  2.3234987   0.21957426  0.3159482   0.7972845
  -1.1508235   1.3413458   0.83222246  0.5255313 ]]...


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 3495373.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 299706.8438, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 14753.2412, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 3852.6289, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 1498.2699, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1612.4880, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 1393.4733, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 1211.7590, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1708.2153, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1392.5122, LR: 1.000e-03

Training complete!
Final Loss: 0.06501167267560959
Final parameters: [[ 1.6859447   0.13510853  1.7694694   0.54262686  0.61068016  0.82488966
  -0.34569275  1.2157702   0.7859552   0.71407807]]...


In [25]:
shutil.rmtree(f"Quadratic/Discount/Train_2", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree(f"Quadratic/Discount/Test_2", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)
n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

initializer = Param_Initializer(QuadraticOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"Quadratic/Discount/Train_2")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=50, max_horizon=50, discount=2, writer=writer)
writer = SummaryWriter(f"Quadratic/Discount/Test_2")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.W = torch.tensor(W, dtype=torch.float32)
c:\Users\miche\Documents\GitHub\Learning-to-Optimize\src\optimizee.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.theta0 = torch.tensor(theta0, dtype=torch.float32)


Epoch [10/100], Cumulative Loss: 91058821413208064.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 122737228878184448.0000, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 105265430767075328.0000, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 188573854845632512.0000, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 76045557071609856.0000, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 74857964254527488.0000, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 58383139721969664.0000, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 48512647745765376.0000, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 74706257419698176.0000, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 70099801915523072.0000, LR: 1.000e-03

Training complete!
Final Loss: 332522.28125
Final parameters: [[ 84.53091   83.46353   79.701775  65.80607   82.73815  129.2053
   99.99969   85.89128   84.35818   90.16891 ]]...


In [7]:
times_train, times_test = [], []
for count in range(1, 10):
    shutil.rmtree(f"Quadratic/Multi/Train_{count}_optim", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Quadratic/Multi/Test_{count}_optim", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)

    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": list(np.arange(0.01, (count+1)*0.01, 0.01))}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    t0 = time.time()
    writer = SummaryWriter(f"Quadratic/Multi/Train_{count}_optim") 
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=200, max_horizon=500, discount=0, writer=writer)
    t1 = time.time()
    writer = SummaryWriter(f"Quadratic/Multi/Test_{count}_optim")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)
    t2 = time.time()

    times_train.append(t1-t0)
    times_test.append(t2-t1)

print("Times_Train", times_train)
print("Times_Test", times_test)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 18863.7344, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.0161, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 9.6801, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0483, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0059, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0231, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0157, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0169, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0197, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0495, LR: 1.000e-03

Training complete!
Final Loss: 0.025036029517650604


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 60361.3008, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.4575, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.7181, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 4.3570, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 3.8612, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 6.1974, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 7.0903, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 2.7111, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 3.8155, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 5.6945, LR: 1.000e-03

Training complete!
Final Loss: 4.595547676086426


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [10/100], Cumulative Loss: 0.7460, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.1576, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0994, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0870, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0555, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0388, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0352, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0820, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0270, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0340, LR: 1.000e-03

Training complete!
Final Loss: 0.10253313928842545


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 0.7460, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.1576, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0994, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0870, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0555, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0388, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0352, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0820, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0270, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0340, LR: 1.000e-03

Training complete!
Final Loss: 0.10253313928842545


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8.0472, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 5.3325, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 3.7232, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 3.0543, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 2.3280, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 2.6844, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 4.1541, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 2.5754, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.5171, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 5.7459, LR: 1.000e-03

Training complete!
Final Loss: 1.1009637117385864
Times_Train [56.72163724899292, 81.90180659294128, 106.4897084236145, 135.35498309135437, 155.31620001792908, 180.24078154563904, 210.21417117118835, 212.05891036987305, 228.83933758735657]
Times_Test [0.8150696754455566, 0.954148530960083, 1.4411990642547607, 1.7495684623718262, 2.428520441055298, 2.3750107288360596, 2.845968723297119, 3.0905141830444336, 2.404212236404419]


In [30]:
for i in range(1,5):

    shutil.rmtree(f"Quadratic/Lr/Train_e{i}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Quadratic/Lr/Test_e{i}", ignore_errors=True)  # Remove the directory if it exists
    lr = 10**(-i)

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)
    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=lr)

    writer = SummaryWriter(f"Quadratic/Lr/Train_e{i}")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=200, min_horizon=500, max_horizon=500, discount=0, writer=writer)
    writer = SummaryWriter(f"Quadratic/Lr/Test_e{i}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 11530.6094, LR: 1.000e-01
Epoch [40/200], Cumulative Loss: 0.4383, LR: 1.000e-01
Epoch [60/200], Cumulative Loss: 0.6987, LR: 1.000e-01
Epoch [80/200], Cumulative Loss: 0.8164, LR: 1.000e-01
Epoch [100/200], Cumulative Loss: 2.5323, LR: 1.000e-01
Epoch [120/200], Cumulative Loss: 0.5414, LR: 1.000e-01
Epoch [140/200], Cumulative Loss: 0.4951, LR: 1.000e-01
Epoch [160/200], Cumulative Loss: 0.2133, LR: 1.000e-01
Epoch [180/200], Cumulative Loss: 0.6893, LR: 1.000e-01
Epoch [200/200], Cumulative Loss: 0.6405, LR: 1.000e-01

Training complete!
Final Loss: 1.2033077478408813
Final parameters: [[-1.993716   4.672194  -2.7682881  3.217623   2.6500154  1.6340405
   7.3981104 -0.326854   1.8212482  2.4724944]]...


Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 3.0271, LR: 1.000e-02
Epoch [40/200], Cumulative Loss: 23.6708, LR: 1.000e-02
Epoch [60/200], Cumulative Loss: 5.9904, LR: 1.000e-02
Epoch [80/200], Cumulative Loss: 9.8778, LR: 1.000e-02
Epoch [100/200], Cumulative Loss: 4.2154, LR: 1.000e-02
Epoch [120/200], Cumulative Loss: 5.8195, LR: 1.000e-02
Epoch [140/200], Cumulative Loss: 3.6592, LR: 1.000e-02
Epoch [160/200], Cumulative Loss: 4.2677, LR: 1.000e-02
Epoch [180/200], Cumulative Loss: 2.6112, LR: 1.000e-02
Epoch [200/200], Cumulative Loss: 2.8854, LR: 1.000e-02

Training complete!
Final Loss: 1.7332725524902344
Final parameters: [[1.5985692  0.06193918 0.80853343 0.86137754 0.84611106 0.70785636
  0.38167152 1.3204658  0.24488646 1.0086565 ]]...


Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 1679.9042, LR: 1.000e-03
Epoch [40/200], Cumulative Loss: 0.0149, LR: 1.000e-03
Epoch [60/200], Cumulative Loss: 0.1046, LR: 1.000e-03
Epoch [80/200], Cumulative Loss: 0.3084, LR: 1.000e-03
Epoch [100/200], Cumulative Loss: 0.1007, LR: 1.000e-03
Epoch [120/200], Cumulative Loss: 0.1111, LR: 1.000e-03
Epoch [140/200], Cumulative Loss: 0.1002, LR: 1.000e-03
Epoch [160/200], Cumulative Loss: 0.1151, LR: 1.000e-03
Epoch [180/200], Cumulative Loss: 0.1102, LR: 1.000e-03
Epoch [200/200], Cumulative Loss: 0.1093, LR: 1.000e-03

Training complete!
Final Loss: 0.07103900611400604
Final parameters: [[ 1.5654256   0.30250198  1.5598316   0.60075116  0.7088844   0.81796616
  -0.05125184  1.1672177   0.7796621   0.8026926 ]]...


Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 151222.4062, LR: 1.000e-04
Epoch [40/200], Cumulative Loss: 104470.3438, LR: 1.000e-04
Epoch [60/200], Cumulative Loss: 65506.3711, LR: 1.000e-04
Epoch [80/200], Cumulative Loss: 36242.8086, LR: 1.000e-04
Epoch [100/200], Cumulative Loss: 17056.4102, LR: 1.000e-04
Epoch [120/200], Cumulative Loss: 7074.2148, LR: 1.000e-04
Epoch [140/200], Cumulative Loss: 2590.7512, LR: 1.000e-04
Epoch [160/200], Cumulative Loss: 719.8875, LR: 1.000e-04
Epoch [180/200], Cumulative Loss: 240.4924, LR: 1.000e-04
Epoch [200/200], Cumulative Loss: 77.0269, LR: 1.000e-04

Training complete!
Final Loss: 95.14395904541016
Final parameters: [[ 1.1561422  -3.5963435  -1.160581   -2.333385    0.44913736 -2.902987
  -1.7678744  -0.38950413 -2.892629    0.25393853]]...


In [35]:
shutil.rmtree("Quadratic/Lr/Train_Exp", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("Quadratic/Lr/Test_Exp", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)
n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

initializer = Param_Initializer(QuadraticOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.01)

scheduler = torch.optim.lr_scheduler.ExponentialLR(meta_optimizer, gamma=0.99)

writer = SummaryWriter(f"Quadratic/Lr/Train_Exp")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=200, min_horizon=500, max_horizon=500, discount=0, writer=writer, scheduler=scheduler)
writer = SummaryWriter(f"Quadratic/Lr/Test_Exp")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 3.0597, LR: 8.179e-03
Epoch [40/200], Cumulative Loss: 20.0837, LR: 6.690e-03
Epoch [60/200], Cumulative Loss: 4.8919, LR: 5.472e-03
Epoch [80/200], Cumulative Loss: 8.1621, LR: 4.475e-03
Epoch [100/200], Cumulative Loss: 3.3011, LR: 3.660e-03
Epoch [120/200], Cumulative Loss: 5.1842, LR: 2.994e-03
Epoch [140/200], Cumulative Loss: 6.6976, LR: 2.449e-03
Epoch [160/200], Cumulative Loss: 2.6100, LR: 2.003e-03
Epoch [180/200], Cumulative Loss: 4.3614, LR: 1.638e-03
Epoch [200/200], Cumulative Loss: 3.0451, LR: 1.340e-03

Training complete!
Final Loss: 1.4719452857971191
Final parameters: [[ 2.814364   -1.4750608   3.383005   -0.14283155 -0.34546745  0.6081619
  -3.4897394   1.8527266   0.3712343  -0.02309484]]...


In [42]:
shutil.rmtree("Quadratic/Comparison/Train_Right_Horizon", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("Quadratic/Comparison/Test_Right_Horizon", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(0)  # Set random seed for reproducibility
np.random.seed(0)
n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1)  # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01]}

initializer = Param_Initializer(QuadraticOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"Quadratic/Comparison/Train_Right_Horizon")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=800, discount=0, writer=writer)
writer = SummaryWriter(f"Quadratic/Comparison/Test_Right_Horizon")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 43108.0234, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.0079, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0405, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0206, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0381, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0365, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0380, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0392, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0617, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0276, LR: 1.000e-03

Training complete!
Final Loss: 0.051029056310653687
Final parameters: [[1.4419096  0.47065443 1.5104656  0.66466796 0.77859986 0.8647847
  0.10427374 1.1317858  0.9094761  0.7959955 ]]...


In [ ]:
torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)
n=10
W = torch.randn(n, n)  # Random weights for the linear model
theta0 = torch.ones(n,1) # Random theta for the linear model

kwargs = {"W": [W], "theta0": [theta0], "noise_std": [0.01, 0.1, 0.2, 0.3, 0.4]}

initializer = Param_Initializer(QuadraticOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent_Intepretable(num_optims=initializer.get_num_optims(), preproc=False)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.01)

lstm_optimizer, lambdas_ = train_LSTM_Intepret(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, time_horizon=50, discount=0.9)
params = test_LSTM_Interpret(lstm_optimizer, initializer, time_horizon=50)

v = nn.utils.parameters_to_vector(lstm_optimizer.input_layer.parameters())
print(nn.functional.softmax(v, dim=0))

In [ ]:
for count in [3, 5, 7]:
    shutil.rmtree(f"Preproc/Quadratic/Train_{count}_Classic", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Preproc/Quadratic/Test_{count}_Classic", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)

    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": list(np.arange(0.01, (count+1)*0.01, 0.01))}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"Preproc/Quadratic/Train_{count}_Classic")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=200, max_horizon=500, discount=0, writer=writer)

    writer = SummaryWriter(f"Preproc/Quadratic/Test_{count}_Classic")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)


In [ ]:
for count in [3, 5, 7]:
    shutil.rmtree(f"Preproc/Quadratic/Train_{count}_Alt1", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Preproc/Quadratic/Test_{count}_Alt1", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)

    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": list(np.arange(0.01, (count+1)*0.01, 0.01))}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"Preproc/Quadratic/Train_{count}_Alt1")
    lstm_optimizer = train_LSTM_alt_preproc(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=200, max_horizon=500, discount=0, writer=writer)

    writer = SummaryWriter(f"Preproc/Quadratic/Test_{count}_Alt1")
    params = test_LSTM_alt_preproc(lstm_optimizer, initializer, time_horizon=500, writer=writer)


In [ ]:
for count in [3, 5, 7]:
    shutil.rmtree(f"Preproc/Quadratic/Train_{count}_Alt2", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"Preproc/Quadratic/Test_{count}_Alt2", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(0)  # Set random seed for reproducibility
    np.random.seed(0)

    n=10
    W = torch.randn(n, n)  # Random weights for the linear model
    theta0 = torch.ones(n,1)  # Random theta for the linear model

    kwargs = {"W": [W], "theta0": [theta0], "noise_std": list(np.arange(0.01, (count+1)*0.01, 0.01))}

    initializer = Param_Initializer(QuadraticOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent_Preproc(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"Preproc/Quadratic/Train_{count}_Alt2")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=200, max_horizon=500, discount=0, writer=writer)

    writer = SummaryWriter(f"Preproc/Quadratic/Test_{count}_Alt2")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)


## NN Optimizee - Single Optim

In [ ]:
shutil.rmtree("NNDiabetes/Train_LSTM", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NNDiabetes/Test_LSTM", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hiddaen_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

lstm_optimizer = LSTMConcurrent(num_optims=1, preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)
initializer = Param_Initializer(XYNNOptimizee, kwargs)

writer = SummaryWriter("NNDiabetes/Train_LSTM") 
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter("NNDiabetes/Test_LSTM")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8006432.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 9.7166, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 2.0988, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.7331, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.7267, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.0989, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.4511, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.5665, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6034, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.1383, LR: 1.000e-03

Training complete!
Final Loss: 0.5529805421829224


In [7]:
for discount in [0, 1e-3, 0.1, 0.9, 1]:
    
    shutil.rmtree(f"NN_Diabetes/Discount_v2/Train_{discount}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"NN_Diabetes/Discount_v2/Test_{discount}", ignore_errors=True)  # Remove the directory if it exists

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)

    kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

    initializer = Param_Initializer(XYNNOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"NN_Diabetes/Discount_v2/Train_{discount}")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=discount, writer=writer)
    writer = SummaryWriter(f"NN_Diabetes/Discount_v2/Test_{discount}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8006432.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 9.7166, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 2.0988, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.7331, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.7267, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.0989, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.4511, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.5665, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6034, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.1383, LR: 1.000e-03

Training complete!
Final Loss: 0.5529805421829224


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8014993.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 9.9052, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.6903, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.5635, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.4246, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.8011, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.5069, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.7418, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6496, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.4590, LR: 1.000e-03

Training complete!
Final Loss: 0.48407047986984253


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8940307.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 2.8788, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.7551, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.5931, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.6959, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.7834, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.7119, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.7548, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.9785, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.6719, LR: 1.000e-03

Training complete!
Final Loss: 0.6535630822181702


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 75164032.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 60.7986, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 12.4487, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 6.1888, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 6.6519, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 7.0342, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 5.9492, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 5.9315, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 8.2504, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 5.9288, LR: 1.000e-03

Training complete!
Final Loss: 0.7123647332191467


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 859471296.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1441.3616, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 436.0175, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 318.4882, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 291.7909, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 349.0607, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 384.4711, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 466.8342, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 308.0726, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 659.2723, LR: 1.000e-03

Training complete!
Final Loss: 0.4745386242866516


In [55]:
for i in range(1,5):

    shutil.rmtree(f"NN_Diabetes/Lr/Train_e{i}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"NN_Diabetes/Lr/Test_e{i}", ignore_errors=True)  # Remove the directory if it exists
    lr = 10**(-i)

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)

    kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

    initializer = Param_Initializer(XYNNOptimizee, kwargs)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=lr)

    writer = SummaryWriter(f"NN_Diabetes/Lr/Train_e{i}")
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=200, min_horizon=500, max_horizon=500, discount=0, writer=writer)
    writer = SummaryWriter(f"NN_Diabetes/Lr/Test_e{i}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 6249670.0000, LR: 1.000e-01
Epoch [40/200], Cumulative Loss: 6372057.5000, LR: 1.000e-01
Epoch [60/200], Cumulative Loss: 5327980.5000, LR: 1.000e-01
Epoch [80/200], Cumulative Loss: 5257824.0000, LR: 1.000e-01
Epoch [100/200], Cumulative Loss: 5160894.5000, LR: 1.000e-01
Epoch [120/200], Cumulative Loss: 5063762.0000, LR: 1.000e-01
Epoch [140/200], Cumulative Loss: 4940141.0000, LR: 1.000e-01
Epoch [160/200], Cumulative Loss: 4815151.0000, LR: 1.000e-01
Epoch [180/200], Cumulative Loss: 4694249.5000, LR: 1.000e-01
Epoch [200/200], Cumulative Loss: 4556052.5000, LR: 1.000e-01

Training complete!
Final Loss: 4577862.5
Final parameters: [[-2139.6338 -2139.5386 -2139.3198 -2138.8416 -2137.8494 -2135.9512
  -2138.3613 -2137.4475 -2138.4233 -2136.694  -2139.2332 -2139.5715
  -2139.9653 -2136.1748 -2138.9194 -2138.7114 -2138.079  -2138.4326
  -2137.951  -2137.8035 -2139.09   -2140.0803 -2136.8918 -2139.1582
  -2139.0652 -2138.2954 -2137.416  -2137.475  -2138.

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 14194.9180, LR: 1.000e-02
Epoch [40/200], Cumulative Loss: 21041.3184, LR: 1.000e-02
Epoch [60/200], Cumulative Loss: 21892.2852, LR: 1.000e-02
Epoch [80/200], Cumulative Loss: 21427.7305, LR: 1.000e-02
Epoch [100/200], Cumulative Loss: 21263.6230, LR: 1.000e-02
Epoch [120/200], Cumulative Loss: 21721.2500, LR: 1.000e-02
Epoch [140/200], Cumulative Loss: 20845.8867, LR: 1.000e-02
Epoch [160/200], Cumulative Loss: 20414.2422, LR: 1.000e-02
Epoch [180/200], Cumulative Loss: 20626.1328, LR: 1.000e-02
Epoch [200/200], Cumulative Loss: 19850.9141, LR: 1.000e-02

Training complete!
Final Loss: 20628.640625
Final parameters: [[-144.21327 -144.17206 -144.6734  -143.44218 -142.86865 -140.9704
  -143.38077 -142.70139 -143.10115 -142.27995 -144.42044 -144.14645
  -144.47533 -141.77798 -143.93872 -143.94342 -142.59561 -143.60626
  -142.97029 -142.82271 -143.40091 -144.40218 -142.87471 -143.45876
  -144.08455 -143.3148  -142.4353  -143.44862 -142.75139 -142.9768
  -

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 7.5716, LR: 1.000e-03
Epoch [40/200], Cumulative Loss: 0.5253, LR: 1.000e-03
Epoch [60/200], Cumulative Loss: 0.8197, LR: 1.000e-03
Epoch [80/200], Cumulative Loss: 0.8909, LR: 1.000e-03
Epoch [100/200], Cumulative Loss: 0.8395, LR: 1.000e-03
Epoch [120/200], Cumulative Loss: 0.4654, LR: 1.000e-03
Epoch [140/200], Cumulative Loss: 0.7502, LR: 1.000e-03
Epoch [160/200], Cumulative Loss: 0.5141, LR: 1.000e-03
Epoch [180/200], Cumulative Loss: 0.6688, LR: 1.000e-03
Epoch [200/200], Cumulative Loss: 0.9374, LR: 1.000e-03

Training complete!
Final Loss: 0.5737376809120178
Final parameters: [[ 2.9833179   3.8622794  -0.14068198  4.3857355   3.7798533   5.3756747
   3.5110707   3.1401856   4.829986    6.0724025   1.8579866   3.8900626
   4.009493    4.063535    2.226865    3.2519507   5.548273    2.7012928
   5.7696447   3.7616239   3.0303867   4.399092    1.5447174   5.156945
   2.9124982   4.1088667   2.3230379   1.7117053   5.8028502   6.3108554
   2.903716

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 823756864.0000, LR: 1.000e-04
Epoch [40/200], Cumulative Loss: 317365088.0000, LR: 1.000e-04
Epoch [60/200], Cumulative Loss: 154674672.0000, LR: 1.000e-04
Epoch [80/200], Cumulative Loss: 85397240.0000, LR: 1.000e-04
Epoch [100/200], Cumulative Loss: 51343612.0000, LR: 1.000e-04
Epoch [120/200], Cumulative Loss: 35554128.0000, LR: 1.000e-04
Epoch [140/200], Cumulative Loss: 23031468.0000, LR: 1.000e-04
Epoch [160/200], Cumulative Loss: 16873882.0000, LR: 1.000e-04
Epoch [180/200], Cumulative Loss: 10926652.0000, LR: 1.000e-04
Epoch [200/200], Cumulative Loss: 9881110.0000, LR: 1.000e-04

Training complete!
Final Loss: 9630429.0
Final parameters: [[14.98499   15.269319  14.378924  16.08163   16.045311  17.987875
  15.2940855 16.236506  16.450212  16.892769  14.446253  15.641608
  14.664139  16.71841   14.998168  14.882536  16.688196  15.23445
  16.40212   16.365612  17.0328    16.229595  17.818953  17.301712
  16.433405  17.172335  17.83573   17.489338 

In [56]:
torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

lstm_optimizer = LSTMConcurrent(num_optims=1, preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.0001)
initializer = Param_Initializer(XYNNOptimizee, kwargs)

writer = SummaryWriter("NN_Diabetes/Lr/Train_e4_Long") 
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=2000, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter("NN_Diabetes/Lr/Test_e4_Long")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/2000 [00:00<?, ?it/s]

Epoch [200/2000], Cumulative Loss: 9881110.0000, LR: 1.000e-04
Epoch [400/2000], Cumulative Loss: 1246417.8750, LR: 1.000e-04
Epoch [600/2000], Cumulative Loss: 382936.7188, LR: 1.000e-04
Epoch [800/2000], Cumulative Loss: 148605.0156, LR: 1.000e-04
Epoch [1000/2000], Cumulative Loss: 72151.5234, LR: 1.000e-04
Epoch [1200/2000], Cumulative Loss: 32419.0684, LR: 1.000e-04
Epoch [1400/2000], Cumulative Loss: 22371.1641, LR: 1.000e-04
Epoch [1600/2000], Cumulative Loss: 4474.1758, LR: 1.000e-04
Epoch [1800/2000], Cumulative Loss: 2441.9375, LR: 1.000e-04
Epoch [2000/2000], Cumulative Loss: 1084.0387, LR: 1.000e-04

Training complete!
Final Loss: 1392.8455810546875
Final parameters: [[ 4.4197307   9.92952     5.375772    7.0339346  20.885983    4.8713417
   2.7107177   4.1928186   4.885844    8.907608    4.54261    21.969252
   4.549441    8.206536    5.313111   20.189985    5.13701     6.5987716
   6.24627     5.765709    5.3153024  12.793395    5.91939     6.311839
  17.92428     6.32871

In [ ]:
shutil.rmtree("NN_Diabetes/Lr/Train_Step", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes/Lr/Test_Step", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.StepLR(meta_optimizer, step_size=40, gamma=np.sqrt(0.1))

writer = SummaryWriter(f"NN_Diabetes/Lr/Train_Step")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=200, min_horizon=500, max_horizon=500, discount=0, writer=writer, scheduler=scheduler)
writer = SummaryWriter(f"NN_Diabetes/Lr/Test_Step")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/200 [00:00<?, ?it/s]

Epoch [20/200], Cumulative Loss: 7.5716, LR: 1.000e-03
Epoch [40/200], Cumulative Loss: 0.5253, LR: 3.162e-04
Epoch [60/200], Cumulative Loss: 0.5635, LR: 3.162e-04
Epoch [80/200], Cumulative Loss: 0.6683, LR: 1.000e-04
Epoch [100/200], Cumulative Loss: 0.4577, LR: 1.000e-04
Epoch [120/200], Cumulative Loss: 0.5452, LR: 3.162e-05
Epoch [140/200], Cumulative Loss: 0.5339, LR: 3.162e-05
Epoch [160/200], Cumulative Loss: 0.7056, LR: 1.000e-05
Epoch [180/200], Cumulative Loss: 1.8736, LR: 1.000e-05
Epoch [200/200], Cumulative Loss: 0.3772, LR: 3.162e-06

Training complete!
Final Loss: 0.6363527774810791
Final parameters: [[ 2.8847816   5.201524    0.12562682  5.690243    3.7675788   5.326632
   3.0322957   3.411771    6.1306696   7.198477    1.9685651   5.1859236
   5.302169    4.1409636   1.8312805   3.2916188   6.9305086   2.8173516
   7.182851    3.118078    3.0049963   5.6659985   1.7903786   6.3867702
   3.0520074   4.2430196   3.684196    2.0288446   7.0280037   7.519516
   3.136249 

In [65]:
shutil.rmtree("NN_Diabetes/Comparison/Train_Right_Horizon", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes/Comparison/Test_Right_Horizon", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"NN_Diabetes/Comparison/Train_Right_Horizon")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=800, discount=0, writer=writer)
writer = SummaryWriter(f"NN_Diabetes/Comparison/Test_Right_Horizon")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 49678488.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 64455.5508, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 3.5796, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.0915, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 1.1040, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 2.4050, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 1.4086, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.7440, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.8023, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.3376, LR: 1.000e-03

Training complete!
Final Loss: 0.8964078426361084
Final parameters: [[ 4.333922   15.034754   13.66389    14.237004    3.0719428   2.0987694
   3.0590067   3.9937625  14.728879    1.8864397   5.748864    1.6719899
   3.6101408   3.6434765   3.923642    3.968352   15.279895    5.018868
   2.9828792   3.8431978   3.0129983  14.562385   14.268546   13.784624
   2.5398457   1.4271225   3.5619824   6.438787   12.611475    2.3104882
   6.3

## NN Optimizee - Multi Optim

In [4]:
shutil.rmtree("NNDiabetes_Multi/Train_Dir_11", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NNDiabetes_Multi/Test_Dir_11", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": X, "y": y, "hidden_size": 20, "num_layers": 2, "num_samples":100, "loss_fn":nn.MSELoss()}

initializer = Data_Initializer(XYNNOptimizee, kwargs, num_optims=10, subset_size=40)
print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter("NNDiabetes_Multi/Train_Dir_11") 
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter("NNDiabetes_Multi/Test_Dir_11")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

10


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 11.6563, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.0274, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.0227, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.0496, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.0429, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.0224, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.0228, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.0287, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.0519, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.0221, LR: 1.000e-03

Training complete!
Final Loss: 0.055267542600631714


In [3]:
for i in np.arange(40, 140, 20):
    shutil.rmtree(f"NNDiabetes_Multi/SubSize/Train_{i}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"NNDiabetes_Multi/SubSize/Test_{i}", ignore_errors=True)  # Remove the directory if it exists

    kwargs = {"X": X, "y": y, "hidden_size": 20, "num_layers": 2, "num_samples":100, "loss_fn":nn.MSELoss()}

    initializer = Data_Initializer(XYNNOptimizee, kwargs, num_optims=5, subset_size=i)

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter(f"NNDiabetes_Multi/SubSize/Train_{i}") 
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
    writer = SummaryWriter(f"NNDiabetes_Multi/SubSize/Test_{i}")
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 9.3712, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.6942, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.7742, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.0797, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.9257, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.8957, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.6777, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.8461, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.2062, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.3523, LR: 1.000e-03

Training complete!
Final Loss: 1.2804855108261108


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 1.8720, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.0809, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.8563, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.5012, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.7889, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.0272, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.6731, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.6559, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.9176, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.5971, LR: 1.000e-03

Training complete!
Final Loss: 0.5302165746688843


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 0.8952, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.2487, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.8769, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.5273, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.8627, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.7031, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.8705, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.6246, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6137, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.5438, LR: 1.000e-03

Training complete!
Final Loss: 0.8982424139976501


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 0.7295, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.2545, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 1.9127, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.0445, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.8711, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.5838, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.4326, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.7885, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.3083, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.0391, LR: 1.000e-03

Training complete!
Final Loss: 0.7224278450012207


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 2.6271, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 3.6102, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.9125, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.6189, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.8111, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.6105, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.9432, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.4965, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6080, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.5315, LR: 1.000e-03

Training complete!
Final Loss: 0.6149126887321472


In [6]:
for i in np.arange(0.6,0.8,0.2):
    i = float(str(f"{i:.2f}"))
    i_str = str(i).replace(".", "")
    shutil.rmtree(f"NNDiabetes_Multi/Dirichlet/Train_Dir_{i_str}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"NNDiabetes_Multi/Dirichlet/Test_Dir_{i_str}", ignore_errors=True)  # Remove the directory if it exists

    kwargs = {"X": X, "y": y, "hidden_size": 20, "num_layers": 2, "num_samples":100, "loss_fn":nn.MSELoss()}
    dist = Dirichlet(torch.tensor([i, i]))

    initializer = Data_Initializer(XYNNOptimizee, kwargs, distribution=dist, num_optims=5, subset_size=60)

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter("NNDiabetes_Multi/Dirichlet/Train_Dir_" + i_str) 
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
    writer = SummaryWriter("NNDiabetes_Multi/Dirichlet/Test_Dir_" + i_str)
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 2.4605, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.4844, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.9941, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.6039, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.4922, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.6912, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.8025, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 1.5898, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.0929, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.0338, LR: 1.000e-03

Training complete!
Final Loss: 1.0842245817184448


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 21.8032, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.4232, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.9033, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.5923, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.5541, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.7923, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 9.1531, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 4.8190, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.7431, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 4.5322, LR: 1.000e-03

Training complete!
Final Loss: 0.9238519072532654


In [7]:
for i in np.arange(0.25,2.25,0.25):
    i = float(str(f"{i:.3f}"))
    i_str = str(i).replace(".", "")
    shutil.rmtree(f"NNDiabetes_Multi/Dirichlet2/Train_Dir_{i_str}", ignore_errors=True)  # Remove the directory if it exists
    shutil.rmtree(f"NNDiabetes_Multi/Dirichlet2/Test_Dir_{i_str}", ignore_errors=True)  # Remove the directory if it exists

    kwargs = {"X": X, "y": y, "hidden_size": 20, "num_layers": 2, "num_samples":100, "loss_fn":nn.MSELoss()}
    dist = Dirichlet(torch.tensor([i, i]))

    initializer = Data_Initializer(XYNNOptimizee, kwargs, distribution=dist, num_optims=5, subset_size=60)

    torch.manual_seed(1)  # Set random seed for reproducibility
    np.random.seed(1)

    lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
    meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

    writer = SummaryWriter("NNDiabetes_Multi/Dirichlet2/Train_Dir_" + i_str) 
    lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
    writer = SummaryWriter("NNDiabetes_Multi/Dirichlet2/Test_Dir_" + i_str)
    params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 3.8885, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 3.1569, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.6032, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.3146, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.5266, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.8746, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.8030, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 1.4714, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.5087, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 2.7344, LR: 1.000e-03

Training complete!
Final Loss: 0.6261980533599854


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 2.9358, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.0775, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.6241, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 2.1537, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.7499, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.7232, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.3988, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.9394, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.2381, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.5663, LR: 1.000e-03

Training complete!
Final Loss: 0.9663375020027161


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

...

Epoch [10/100], Cumulative Loss: 1.9181, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 1.2097, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 1.2240, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.4632, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.6835, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.7340, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.5765, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.8840, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.8503, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.4778, LR: 1.000e-03

Training complete!
Final Loss: 0.5290380120277405


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 2.0031, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 13.8098, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 4.0023, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.3252, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 3.0893, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.0997, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.9132, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 1.5231, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.6686, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 2.2863, LR: 1.000e-03

Training complete!
Final Loss: 1.0342798233032227


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 4.2231, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.5239, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.8191, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 1.1897, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.8875, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.8233, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.8568, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 1.3659, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.3039, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.9797, LR: 1.000e-03

Training complete!
Final Loss: 1.1990711688995361


## NN Diab Functions

In [15]:
shutil.rmtree("NN_Diabetes_Multi/Comparison/Tr_Single", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes_Multi/Comparison/Ts_Single", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Tr_Single")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Ts_Single")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

1


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 8006432.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 9.7166, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 2.0988, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.7331, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.7267, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 1.0989, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.4511, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.5665, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.6034, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 1.1383, LR: 1.000e-03

Training complete!
Final Loss: 0.5529805421829224


In [16]:
shutil.rmtree("NN_Diabetes_Multi/Comparison/Tr_Loss", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes_Multi/Comparison/Ts_Loss", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss(), nn.L1Loss(), nn.SmoothL1Loss(), nn.BCEWithLogitsLoss()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Tr_Loss")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Ts_Loss")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

4


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 5844946.0000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 4.8254, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.4726, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.5150, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.4103, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.5669, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.3464, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.3258, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.4290, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.3969, LR: 1.000e-03

Training complete!
Final Loss: 0.3394172787666321


In [17]:
shutil.rmtree("NN_Diabetes_Multi/Comparison/Tr_Activation", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes_Multi/Comparison/Ts_Activation", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()], "activation_fn":[nn.ReLU(), nn.Tanh(), nn.Sigmoid(), nn.SiLU()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Tr_Activation")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Ts_Activation")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

4


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 1334267.5000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 15.4812, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 6.8238, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.7594, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.6183, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.8165, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 5.4469, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 2.4528, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 1.7113, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 3.8405, LR: 1.000e-03

Training complete!
Final Loss: 0.892021119594574


In [18]:
shutil.rmtree("NN_Diabetes_Multi/Comparison/Tr_Both", ignore_errors=True)  # Remove the directory if it exists
shutil.rmtree("NN_Diabetes_Multi/Comparison/Ts_Both", ignore_errors=True)  # Remove the directory if it exists

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss(), nn.L1Loss()], "activation_fn":[nn.ReLU(), nn.Sigmoid()]}

initializer = Param_Initializer(XYNNOptimizee, kwargs)

print(initializer.get_num_optims())

lstm_optimizer = LSTMConcurrent(num_optims=initializer.get_num_optims(), preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)

writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Tr_Both")
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=100, min_horizon=500, max_horizon=500, discount=0, writer=writer)
writer = SummaryWriter(f"NN_Diabetes_Multi/Comparison/Ts_Both")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

4


Training Progress:   0%|          | 0/100 [00:00<?, ?it/s]

Epoch [10/100], Cumulative Loss: 6967467.5000, LR: 1.000e-03
Epoch [20/100], Cumulative Loss: 0.8352, LR: 1.000e-03
Epoch [30/100], Cumulative Loss: 0.6225, LR: 1.000e-03
Epoch [40/100], Cumulative Loss: 0.3854, LR: 1.000e-03
Epoch [50/100], Cumulative Loss: 0.3643, LR: 1.000e-03
Epoch [60/100], Cumulative Loss: 0.5370, LR: 1.000e-03
Epoch [70/100], Cumulative Loss: 0.3581, LR: 1.000e-03
Epoch [80/100], Cumulative Loss: 0.3234, LR: 1.000e-03
Epoch [90/100], Cumulative Loss: 0.3776, LR: 1.000e-03
Epoch [100/100], Cumulative Loss: 0.4049, LR: 1.000e-03

Training complete!
Final Loss: 0.3693506121635437


## Comparison

In [12]:
shutil.rmtree("NN_Comparison/Adam", ignore_errors=True)

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

optimizee = XYNNOptimizee(X,y)
optimizee.set_params()
optimizer_cls = optim.Adam
optimizer_kwargs = {"lr":0.01}

writer = SummaryWriter(f"NN_Comparison/Adam")
optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=writer)
# optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=None)

Training Progress:   0%|          | 0/500 [00:00<?, ?time step/s]

Final training loss: 0.23186102509498596


In [16]:
# shutil.rmtree("NN_Comparison/SGD", ignore_errors=True)

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

optimizee = XYNNOptimizee(X,y)
optimizee.set_params()
optimizer_cls = optim.SGD
optimizer_kwargs = {"lr":0.05}

writer = SummaryWriter(f"NN_Comparison/SGD")
# optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=writer)
optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=None)

Training Progress:   0%|          | 0/500 [00:00<?, ?time step/s]

Final training loss: 0.2659434378147125


In [20]:
# shutil.rmtree("NN_Comparison/MSGD", ignore_errors=True)

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

optimizee = XYNNOptimizee(X,y)
optimizee.set_params()
optimizer_cls = optim.SGD
optimizer_kwargs = {"lr":0.05, "momentum": 0.9}

writer = SummaryWriter(f"NN_Comparison/MSGD")
# optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=writer)
optimizee.train_model(optimizer_cls, optimizer_kwargs, time_horizon=500, writer=None)

Training Progress:   0%|          | 0/500 [00:00<?, ?time step/s]

Final training loss: 0.2353810966014862


In [7]:
shutil.rmtree("NN_Comparison/LSTM", ignore_errors=True)

torch.manual_seed(1)  # Set random seed for reproducibility
np.random.seed(1)

kwargs = {"X": [X], "y": [y], "hidden_size": [20], "num_layers": [2], "num_samples":[100], "loss_fn":[nn.MSELoss()]}

lstm_optimizer = LSTMConcurrent(num_optims=1, preproc=True)
meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.001)
initializer = Param_Initializer(XYNNOptimizee, kwargs)

lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=50, min_horizon=500, max_horizon=500, discount=0)
writer = SummaryWriter("NN_Comparison/LSTM_e3")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

meta_optimizer = optim.Adam(lstm_optimizer.parameters(), lr=0.0001)
initializer = Param_Initializer(XYNNOptimizee, kwargs)
lstm_optimizer = train_LSTM(lstm_optimizer, meta_optimizer, initializer, num_epochs=50, min_horizon=500, max_horizon=500, discount=0)

writer = SummaryWriter("NN_Comparison/LSTM_e4")
params = test_LSTM(lstm_optimizer, initializer, time_horizon=500, writer=writer)

Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [5/50], Cumulative Loss: 213020752.0000, LR: 1.000e-03
Epoch [10/50], Cumulative Loss: 8006432.0000, LR: 1.000e-03
Epoch [15/50], Cumulative Loss: 45590.1992, LR: 1.000e-03
Epoch [20/50], Cumulative Loss: 9.7166, LR: 1.000e-03
Epoch [25/50], Cumulative Loss: 0.8350, LR: 1.000e-03
Epoch [30/50], Cumulative Loss: 2.0988, LR: 1.000e-03
Epoch [35/50], Cumulative Loss: 0.6811, LR: 1.000e-03
Epoch [40/50], Cumulative Loss: 0.7331, LR: 1.000e-03
Epoch [45/50], Cumulative Loss: 0.4565, LR: 1.000e-03
Epoch [50/50], Cumulative Loss: 0.7267, LR: 1.000e-03

Training complete!
Final Loss: 0.9209152460098267


Training Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [5/50], Cumulative Loss: 0.5450, LR: 1.000e-04
Epoch [10/50], Cumulative Loss: 0.6523, LR: 1.000e-04
Epoch [15/50], Cumulative Loss: 0.4074, LR: 1.000e-04
Epoch [20/50], Cumulative Loss: 0.3281, LR: 1.000e-04
Epoch [25/50], Cumulative Loss: 0.4269, LR: 1.000e-04
Epoch [30/50], Cumulative Loss: 0.4663, LR: 1.000e-04
Epoch [35/50], Cumulative Loss: 0.3490, LR: 1.000e-04
Epoch [40/50], Cumulative Loss: 0.3610, LR: 1.000e-04
Epoch [45/50], Cumulative Loss: 0.4363, LR: 1.000e-04
Epoch [50/50], Cumulative Loss: 0.4247, LR: 1.000e-04

Training complete!
Final Loss: 0.42720818519592285
